In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("").setMaster("spark://workspace:7077")
sparkContext = SparkContext(conf=conf)
spark = SparkSession(sparkContext=sparkContext)

24/01/05 15:39:52 WARN Utils: Your hostname, workspace resolves to a loopback address: 127.0.0.1; using 220.118.158.128 instead (on interface eno1)
24/01/05 15:39:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 15:39:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import expr, explode, col

In [4]:
json_dir = "file:///home/hooniegit/git/Spotify-DemoProject/fastapi/data/albums/2023-12-29"

In [5]:
df = spark.read.option("multiline", "true").json(json_dir)
df.printSchema()

root
 |-- albums: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- album_type: string (nullable = true)
 |    |    |-- artists: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |    |    |-- spotify: string (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- uri: string (nullable = true)
 |    |    |-- copyrights: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- external_ids: struct (nullable = true)
 |    |    |    |-- amgid: string (nullable = true)
 |    |    |    |--

In [15]:
df = spark.read.option("multiline", "true").json(json_dir) \
    .withColumn("albums", explode("albums")) \
    .withColumn("images", explode("albums.images")) \
    .filter("images.height = 640") \
    .withColumn("tracks", explode("albums.tracks.items")) \
    .withColumn("artists", explode("albums.artists")) \
    .selectExpr(
        "albums.id",
        "albums.name",
        "images.url as image_url",
        "artists.id as artist_id",
        "tracks.id as track_id",
        "tracks.name as track_name",
        "tracks.duration_ms as duration_ms",
        "tracks.explicit as explicit",
        "tracks.disc_number as disc_number",
        "tracks.track_number as track_number",
    )

df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-----------+------------+
|                  id|                name|           image_url|           artist_id|            track_id|          track_name|duration_ms|explicit|disc_number|track_number|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-----------+------------+
|6Bi5Q5VIzGz0c6zat...|Victoria's Secret...|https://i.scdn.co...|0LyfQWJT6nXafLPZq...|4FU8T0t7YASGlMaA8...|   Victoria's Secret|     176124|    true|          1|           1|
|6Bi5Q5VIzGz0c6zat...|Victoria's Secret...|https://i.scdn.co...|0LyfQWJT6nXafLPZq...|6QEv2OySmn8WAjTlH...|      Left and Right|     154486|   false|          1|           2|
|6Bi5Q5VIzGz0c6zat...|Victoria's Secret...|https://i.scdn.co...|0LyfQWJT6nXafLPZq...|5xBgTb41fF3Pa7KLP...|     About Damn Time|   

In [16]:
spark.stop()